In [1]:

!python --version


Python 3.11.11


In [3]:

## !pip install datasets transformers


In [4]:

import transformers

print(transformers.__version__)


4.48.3


In [5]:

from datasets import load_dataset

datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [6]:

datasets["train"][10]


{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [7]:

from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


In [8]:

show_random_elements(datasets["train"])


,text
0,"Wheeler retained an active interest in the running of these British institutions abroad ; in 1967 he visited the British School in Jerusalem amid the Six @-@ Day War between Israel and its Arab neighbours , and in January 1968 visited the Persian institute with the archaeologist Max Mallowan and Mallowan 's wife Agatha Christie , there inspecting the excavations at Siraf . In 1969 he proceeded to the Italian city of Rome to inspect the British School there . That year , he resigned as Honorary Secretary of the Academy . The position became a salaried , professional one , with the numismatist Derek Allen taking on the position . \n"
1,"The vast majority of pupils at Carre 's continue on to the Sixth Form , and there were 240 students on roll in 2013 . Along with St George 's Academy , Carre 's is part of the Sleaford Joint Sixth Form , which was founded in 1983 and included Kesteven and Sleaford High School until it withdrew in 2010 . It provides a common timetable across both sites and allows for pupils to choose from A @-@ Level options offered at both schools . Pupils may apply to be based at either school , where their pastoral and tutorial activities take place . There are entry requirements based on GCSE attainment . Sixth Formers can be selected to be prefects , who have responsibilities around the school under the supervision of staff ; they are headed by a Head Boy and Girl . The students are required to wear business @-@ dress ; for males , this consists of a dark suit , a "" non @-@ vivid "" shirt and the school 's sixth form tie ; females must dress in "" smart business wear "" . \n"
2,= = Posthumous recognition = = \n
3,"M. Giant of Television Without Pity awarded the episode an "" A "" . Brendan Babish of DVD Verdict gave the episode a moderately positive review and awarded the entry a "" B "" . He wrote that while it was "" "" a solid episode "" it "" lacks any of the belly laughs the show frequently elicits . "" Michael Sciannamea of AOLTV called the installment "" a terrific episode "" and wrote that the "" Michael Scott [ … ] was at his obnoxious best [ … ] in this episode "" . Furthermore , he highly praised the story , noting that "" the Jim / Pam scenario has definitely taken a more interesting turn . "" During the filming of "" The Secret "" , the cast of the show discovered that Carell had been nominated for a Golden Globe Award . Fischer later noted that it was "" fun that [ ' The Secret ' is ] the episode that airs after his win . "" \n"
4,
5,= = = First World War = = = \n
6,
7,
8,
9,"Goffman saw this book as his magnum opus , but it was not as popular as his earlier works . \n"


In [10]:

len(datasets["train"])


36718

In [11]:

model_checkpoint = "distilgpt2"


In [12]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [13]:

def tokenize_function(examples):
    return tokenizer(examples["text"])


In [14]:

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])


Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [18]:

tokenized_datasets["train"][200]


{'input_ids': [8283,
  3905,
  2162,
  7023,
  329,
  262,
  47805,
  286,
  4302,
  20414,
  837,
  37272,
  220,
  198],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:

 tokenizer.model_max_length


1024

In [20]:

block_size = 128


In [41]:

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


In [47]:

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)


In [43]:

tokenizer.decode(lm_datasets["train"][1]["input_ids"])


' game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Oz'

In [44]:

from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [45]:

from transformers import Trainer, TrainingArguments


In [46]:

model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [48]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)


In [50]:


import wandb
wandb.init(mode="disabled")



In [51]:

trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.719400,3.666301
2,3.619500,3.646208
3,3.573300,3.642510


TrainOutput(global_step=7002, training_loss=3.66557739598858, metrics={'train_runtime': 1587.229, 'train_samples_per_second': 35.28, 'train_steps_per_second': 4.411, 'total_flos': 1829011929956352.0, 'train_loss': 3.66557739598858, 'epoch': 3.0})

In [52]:

import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


Perplexity: 38.19
